# Bronze

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, BooleanType, TimestampType

def create_bronze_table(schema,tablename,path):
    df = spark.read.schema(schema).json(path)
    df.write.mode("overwrite").saveAsTable("testing.product_catalog.bronze_{tablename}".format(tablename=tablename))

## Category Tree

In [0]:
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True)
])
path="/Volumes/testing/product_catalog/raw_files/Category_tree.json"
table="category_tree"
create_bronze_table(schema,table,path)

## Category

In [0]:
category_schema = StructType([
    StructField("category_id", IntegerType(), True),
    StructField("tree_id", IntegerType(), True),
    StructField("parent_id", IntegerType(), True), 
    StructField("name", StringType(), True),
    StructField("description", StringType(), True),
    StructField("search_keywords", StringType(), True)
])
path="/Volumes/testing/product_catalog/raw_files/Category.json"
table="category"
create_bronze_table(category_schema,table,path)

## Brand

In [0]:
brand_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("description", StringType(), True)
])
path="/Volumes/testing/product_catalog/raw_files/Brand.json"
table="brand"
create_bronze_table(brand_schema,table,path)

## Product

In [0]:
product_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("type", StringType(), True),
    StructField("sku", StringType(), True),
    StructField("description", StringType(), True),
    StructField("weight", DoubleType(), True),
    StructField("width", DoubleType(), True),
    StructField("depth", DoubleType(), True),
    StructField("height", DoubleType(), True),
    StructField("price", DoubleType(), True),
    StructField("cost_price", DoubleType(), True),
    StructField("retail_price", DoubleType(), True),
    StructField("sale_price", DoubleType(), True),
    StructField("category_id", IntegerType(), True),
    StructField("brand_id", IntegerType(), True),
    StructField("fixed_cost_shipping_price", DoubleType(), True),
    StructField("is_free_shipping", BooleanType(), True),
    StructField("upc", StringType(), True),
    StructField("search_keywords", StringType(), True),
    StructField("availability", StringType(), True),
    StructField("sort_order", IntegerType(), True),
    StructField("order_quantity_minimum", IntegerType(), True),
    StructField("order_quantity_maximum", IntegerType(), True),
    StructField("gtin", StringType(), True),
    StructField("mpn", StringType(), True),
    StructField("reviews_rating_sum", IntegerType(), True),
    StructField("reviews_count", IntegerType(), True),
    StructField("total_sold", IntegerType(), True),
    StructField("date_created", TimestampType(), True),
    StructField("date_modified", TimestampType(), True)
])
path="/Volumes/testing/product_catalog/raw_files/Product.json"
table="product"
create_bronze_table(product_schema,table,path)

## Product variant

In [0]:
sku_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("product_id", IntegerType(), True),
    StructField("sku", StringType(), True),
    StructField("sku_id", IntegerType(), True),
    StructField("cost_price", DoubleType(), True),
    StructField("price", DoubleType(), True),
    StructField("sale_price", DoubleType(), True),
    StructField("retail_price", DoubleType(), True),
    StructField("weight", DoubleType(), True),
    StructField("width", DoubleType(), True),
    StructField("height", DoubleType(), True),
    StructField("depth", DoubleType(), True),
    StructField("is_free_shipping", BooleanType(), True),
    StructField("upc", StringType(), True),
    StructField("mpn", StringType(), True),
    StructField("gtin", StringType(), True)
])
path="/Volumes/testing/product_catalog/raw_files/Product_Variant.json"
table="product_variant"
create_bronze_table(sku_schema,table,path)

# silver

## Category tree

In [0]:
df=spark.table("testing.product_catalog.bronze_category_tree")
df.show()

In [0]:
%sql
create table if not exists testing.product_catalog.silver_category_tree (
SELECT 
  concat('CT', CAST(id AS STRING)) AS id,
  REPLACE(
    REGEXP_REPLACE(
      REPLACE(name, '&', ''),
      r'\s+',
      ' '
    ), 
  ' ', '_') AS name
FROM testing.product_catalog.bronze_category_tree
)

## Category

In [0]:
df=spark.table("testing.product_catalog.bronze_category")
df.show()

In [0]:
%sql
create table if not exists testing.product_catalog.silver_category (
SELECT concat('C', CAST(category_id AS STRING)) AS category_id,
       concat('CT', CAST(tree_id AS STRING)) AS tree_id,
       concat('C', CAST(parent_id AS STRING)) AS parent_id,
       name,
       description,
       search_keywords
FROM testing.product_catalog.bronze_category
)

## Brand

In [0]:
df=spark.table("testing.product_catalog.bronze_brand")
df.show()

In [0]:
%sql
create table if not exists  testing.product_catalog.silver_brand (
SELECT concat('B', CAST(id AS STRING)) AS brand_id,
       name,
       description
FROM testing.product_catalog.bronze_brand
)

## Product

In [0]:
df=spark.table("testing.product_catalog.bronze_product")
df.show(2)

In [0]:
%sql
-- delete from testing.product_catalog.silver_product;
create table if not exists testing.product_catalog.silver_product(
select concat('P',cast(id as string)) as product_id,name,type,sku,description,weight,width,depth,height,price,cost_price,retail_price,sale_price,concat('C',cast(category_id as string)) as category_id,concat('B',cast(brand_id as string)) as brand_id,fixed_cost_shipping_price,is_free_shipping,upc,search_keywords,availability,sort_order,order_quantity_minimum,order_quantity_maximum,gtin,mpn,reviews_rating_sum,reviews_count,total_sold,date_created,date_modified from testing.product_catalog.bronze_product
);

## Product variant

In [0]:
df=spark.table("testing.product_catalog.bronze_product_variant")
df.show(2)

In [0]:
%sql
create table if not exists testing.product_catalog.silver_product_variant(
select concat('PV',cast(id as string)) as prod_var_id,concat('P',cast(product_id as string)) as product_id,sku,sku_id,cost_price,price,sale_price,retail_price,weight,width,height,depth,is_free_shipping,upc,mpn,gtin from testing.product_catalog.bronze_product_variant
)

In [0]:
%sql
select * from testing.product_catalog.silver_product_variant